### Insert a sample and run small experiment

In [ ]:
import mcstasscript as ms
import template

In [ ]:
instrument = template.make(input_path="instrument_code", output_path="data/data", NeXus=True)

## Insert Union sample environment
Here we insert a simple Union sample environment, for now it just contains an empty container.

In [ ]:
sample_position = instrument.get_component("sample_position")

# Union setup, init master and stop, all other components go before master.
init = instrument.add_component("init", "Union_init", after=sample_position)
master = instrument.add_component("master", "Union_master", after=init)
stop = instrument.add_component("stop", "Union_stop", after=master)

Al_inc = instrument.add_component("Al_inc", "Incoherent_process", before=master)
Al_inc.sigma = 0.0082
Al_inc.unit_cell_volume = 66.4

Al_pow = instrument.add_component("Al_pow", "Powder_process", before=master)
Al_pow.reflections = '"Al.laz"'

Al = instrument.add_component("Al", "Union_make_material", before=master)
Al.process_string = '"Al_inc,Al_pow"'
Al.my_absorption = 100*0.231/66.4 # barns [m^2 E-28]*Å^3 [m^3 E-30]=[m E-2], factor 100

sample_volume = instrument.add_component("sample_volume", "Union_cylinder", before=master)
sample_volume.yheight = 0.03
sample_volume.radius = 0.0075
sample_volume.material_string='"Vacuum"' 
sample_volume.priority = 100
sample_volume.set_AT(0, RELATIVE=sample_position)

container = instrument.add_component("sample_container", "Union_cylinder", before=master)
container.set_RELATIVE(sample_volume)
container.yheight = 0.03+0.003 # 1.5 mm top and button
container.radius = 0.0075 + 0.0015 # 1.5 mm sides of container
container.material_string='"Al"' 
container.priority = 99

container_lid = instrument.add_component("sample_container_lid", "Union_cylinder", before=master)
container_lid.set_AT([0, 0.0155, 0], RELATIVE=container)
container_lid.yheight = 0.004
container_lid.radius = 0.013
container_lid.material_string='"Al"' 
container_lid.priority = 98

inner_wall = instrument.add_component("cryostat_wall", "Union_cylinder", before=master)
inner_wall.set_AT([0,0,0], RELATIVE=sample_volume)
inner_wall.yheight = 0.12
inner_wall.radius = 0.03
inner_wall.material_string='"Al"' 
inner_wall.priority = 80

inner_wall_vac = instrument.add_component("cryostat_wall_vacuum", "Union_cylinder", before=master)
inner_wall_vac.set_AT([0,0,0], RELATIVE=sample_volume)
inner_wall_vac.yheight = 0.12 - 0.008
inner_wall_vac.radius = 0.03 - 0.002
inner_wall_vac.material_string='"Vacuum"' 
inner_wall_vac.priority = 81

logger_zx = instrument.add_component("logger_space_zx", "Union_logger_2D_space", before=master)
logger_zx.set_RELATIVE(sample_volume)
logger_zx.D_direction_1 = '"z"'
logger_zx.D1_min = -0.04
logger_zx.D1_max = 0.04
logger_zx.n1 = 300
logger_zx.D_direction_2 = '"x"'
logger_zx.D2_min = -0.04
logger_zx.D2_max = 0.04
logger_zx.n2 = 300
logger_zx.filename = '"logger_zx.dat"'

logger_zy = instrument.add_component("logger_space_zy", "Union_logger_2D_space", before=master)
logger_zy.set_RELATIVE(sample_volume)
logger_zy.D_direction_1 = '"z"'
logger_zy.D1_min = -0.04
logger_zy.D1_max = 0.04
logger_zy.n1 = 300
logger_zy.D_direction_2 = '"y"'
logger_zy.D2_min = -0.06
logger_zy.D2_max = 0.06
logger_zy.n2 = 300
logger_zy.filename = '"logger_zy.dat"'

## See new diagram
Lets see the new diagram and check things are in the right order.

In [ ]:
instrument.show_diagram()

### Run the instrument without sample

In [ ]:
instrument.settings(ncount=5E7, mpi=4, suppress_output=True)
#instrument.set_parameters(l_min=1.5, l_max=1.6, chopper_wavelength_center=1.55)
data = instrument.backengine()

In [ ]:
ms.make_sub_plot(data, log=True, orders_of_mag=6)

### Histogram event data
The detectors use event mode, so they need to be histogramed before they can be plotted.

In [ ]:
Banana_large = ms.name_search("Banana_large", data)
banana_large_hist = Banana_large.make_2d("th", "y", n_bins=[160, 20])
Banana_small = ms.name_search("Banana_small", data)
banana_small_hist = Banana_small.make_2d("th", "y", n_bins=[30, 20])

ms.make_sub_plot([banana_large_hist, banana_small_hist], log=False)

### Histogram as 1d as well

In [ ]:
Banana_large = ms.name_search("Banana_large", data)
banana_large_hist = Banana_large.make_1d("th", n_bins=160)
Banana_small = ms.name_search("Banana_small", data)
banana_small_hist = Banana_small.make_1d("th", n_bins=30)

ms.make_sub_plot([banana_large_hist, banana_small_hist], log=False)

### Now we add a sample
We define Cu and set it as the sample volume, then rerun the instrument with the same settings.

In [ ]:
Cu_incoherent = instrument.add_component("Cu_incoherent", "Incoherent_process", before="sample_volume")
Cu_incoherent.set_parameters(sigma=4 * 0.55, unit_cell_volume=47.24)

Cu_powder = instrument.add_component("Cu_powder", "Powder_process", before="sample_volume")
Cu_powder.reflections = '"Cu.laz"'

Cu = instrument.add_component("Cu", "Union_make_material", before="sample_volume")
Cu.process_string = '"Cu_incoherent,Cu_powder"'
Cu.my_absorption = 100 * 4 * 3.78 / 47.24

sample_volume.material_string = '"Cu"'

In [ ]:
instrument.settings(ncount=5E7, mpi=4, suppress_output=True)
data_sample = instrument.backengine()

In [ ]:
Banana_large_sample = ms.name_search("Banana_large", data_sample)
banana_large_hist_sample = Banana_large_sample.make_1d("th", n_bins=160)
Banana_small_sample = ms.name_search("Banana_small", data_sample)
banana_small_hist_sample = Banana_small_sample.make_1d("th", n_bins=30)

ms.make_sub_plot([banana_large_hist_sample, banana_small_hist_sample], log=False)

### Compare with and without sample
In order to compare the two runs directly, we can use matplotlib to make a custom plot.

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.plot(banana_large_hist_sample.xaxis, banana_large_hist_sample.Intensity, label="with sample")
plt.plot(banana_large_hist.xaxis, banana_large_hist.Intensity, label="without sample")
plt.xlabel(Banana_large.metadata.xlabel)
plt.ylabel(Banana_large.metadata.ylabel)
plt.legend()
plt.show()

### McStasToX
Its also possible to use the newly release McStasToX tool to look at the data with scipp. Needs to be installed first.

In [ ]:
!pip install mcstastox

### Load the data as scipp object
See scipp documentation for explanation, here we plot and do a coordinate transformation to d-spacing.

In [ ]:
import mcstastox
with mcstastox.Read(data_sample[0].original_data_location) as file:
    scipp_object = file.export_scipp_simple("Source", "sample_position")

In [ ]:
scipp_object

In [ ]:
import plopp as pp

pp.scatter3d(scipp_object[0::3], pos='position', size=0.02, cbar=True, norm="linear")

In [ ]:
from scippneutron.conversion.graph.beamline import beamline
from scippneutron.conversion.graph.tof import elastic

# McStas provides absolute time, not time of flight
scipp_object.coords["tof"] = scipp_object.coords["t"]

graph = {**beamline(scatter=True), **elastic("tof")}

In [ ]:
scipp_object = scipp_object.transform_coords("dspacing", graph=graph)

In [ ]:
%matplotlib widget
scipp_object.hist(dspacing=500).plot(norm="linear")